# Analyse Exploratoire des Données (EDA) - Que faire à Paris ?

Ce notebook propose une analyse approfondie des événements culturels et loisirs à Paris, basés sur les données de l'API OpenData Paris.

## Objectifs
1. **Compréhension de la structure** : Identifier les variables clés et la qualité des données.
2. **Nettoyage** : Traiter les valeurs manquantes, les doublons et formater les types de données.
3. **Analyse Temporelle** : Étudier la saisonnalité et la durée des événements.
4. **Analyse Géographique** : Visualiser la répartition spatiale des activités.
5. **Analyse Textuelle** : Explorer les thématiques via les descriptions et les tags.
6. **Analyse des Tarifs** : Comprendre la politique de prix (gratuité vs payant).

In [1]:
# Installation des dépendances si nécessaire
%pip install pandas matplotlib seaborn plotly requests nbformat pymongo --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from pymongo import MongoClient

# Settings for better visualization
sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Chargement et Récupération des Données depuis MongoDB

Nous allons connecter le notebook à la base de données MongoDB hébergée dans le conteneur Docker.
Assurez-vous que le conteneur MongoDB est en cours d'exécution.


In [3]:
def get_data_from_mongo():
    """
    Connects to local MongoDB and retrieves data from events_db.raw_events.
    """
    try:
        # Connection params - connecting to localhost since port 27017 is forwarded
        client = MongoClient(
            host="localhost",
            port=27017,
            username="admin",
            password="password",
            authSource="admin",
            serverSelectionTimeoutMS=5000
        )
        
        # Check connection
        client.admin.command('ping')
        print("Connected to MongoDB successfully.")
        
        db = client['events_db']
        collection = db['raw_events']
        
        # Count documents
        count = collection.count_documents({})
        print(f"Found {count} documents in collection 'raw_events'.")
        
        if count == 0:
            return pd.DataFrame()
        
        # Fetch data (exclude _id for cleaner dataframe)
        cursor = collection.find({}, {'_id': 0})
        data = list(cursor)
        
        client.close()
        return pd.DataFrame(data)
        
    except Exception as e:
        print(f"Error connecting or retrieving data from MongoDB: {e}")
        return pd.DataFrame()

# Load data
df = get_data_from_mongo()

# Display confirmation
print(f"Dataset shape: {df.shape}")


Connected to MongoDB successfully.
Found 0 documents in collection 'raw_events'.
Dataset shape: (0, 0)


## 2. Exploration de la Structure et Nettoyage Initial

Avant toute analyse, il est crucial de comprendre les colonnes disponibles et de normaliser leurs noms si nécessaire.

In [4]:
# List all columns to understand the schema
print("Columns found:", df.columns.tolist())

# Display first few rows
df.head(3)

Columns found: []


""


### Identification des Valeurs Manquantes
Visualisons les données manquantes pour identifier les colonnes exploitables.

In [5]:
if not df.empty and len(df) > 0:
    plt.figure(figsize=(12, 6))
    sns.heatmap(df.isnull(), cbar=False, cmap='viridis', yticklabels=False)
    plt.title("Carte des Valeurs Manquantes (Jaune = Manquant)")
    plt.show()

    # Percentage of missing values per column
    missing_percent = (df.isnull().sum() / len(df)) * 100
    print(missing_percent[missing_percent > 0].sort_values(ascending=False))
else:
    print("DataFrame est vide ou invalide. Impossible de tracer la heatmap.")

DataFrame est vide ou invalide. Impossible de tracer la heatmap.


## 3. Analyse Temporelle

Nous allons convertir les dates et analyser la fréquence des événements par mois et par jour de la semaine.

In [6]:
if not df.empty:
    # Convert date columns
    date_columns = ['date_start', 'date_end']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Feature Engineering: Month, Day of Week, Duration
    if 'date_start' in df.columns:
        df['start_month'] = df['date_start'].dt.month_name()
        df['start_day'] = df['date_start'].dt.day_name()
        df['year'] = df['date_start'].dt.year

        # Calculate duration in days
        if 'date_end' in df.columns:
            df['duration_days'] = (df['date_end'] - df['date_start']).dt.days
            df['duration_days'] = df['duration_days'].fillna(0)  # One day events

In [7]:
# Visualization: Events by Month
if 'start_month' in df.columns:
    plt.figure(figsize=(12, 6))
    order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    valid_order = [m for m in order if m in df['start_month'].unique()]
    sns.countplot(data=df, x='start_month', order=valid_order, palette='coolwarm')
    plt.title("Nombre d'Événements par Mois")
    plt.xticks(rotation=45)
    plt.show()

## 4. Analyse Géographique

Où se déroulent les événements ? Utilisons les coordonnées GPS (lat/lon) si disponibles.

In [8]:
if not df.empty:
    # Clean Lat/Lon data
    geo_cols_candidates = {'lat': ['lat', 'latitude'], 'lon': ['lon', 'long', 'longitude']}

    # Find actual column names for latitude and longitude
    lat_col, lon_col = None, None

    for col in df.columns:
        if col.lower() in geo_cols_candidates['lat']:
            lat_col = col
        if col.lower() in geo_cols_candidates['lon']:
            lon_col = col

    # If columns explicitly named like 'lat_lon' containing a dictionary/list exist
    if not lat_col and 'lat_lon' in df.columns:
        # Attempt to extract if it's a dictionary or object
        try:
            df['lat'] = df['lat_lon'].apply(lambda x: x['lat'] if isinstance(x, dict) else (x[0] if isinstance(x, list) else None))
            df['lon'] = df['lat_lon'].apply(lambda x: x['lon'] if isinstance(x, dict) else (x[1] if isinstance(x, list) else None))
            lat_col, lon_col = 'lat', 'lon'
        except Exception:
            pass

    if lat_col and lon_col:
        # Remove invalid coordinates
        df_geo = df.dropna(subset=[lat_col, lon_col])
        
        print(f"Plotting map with {len(df_geo)} points")
        
        fig = px.scatter_mapbox(df_geo, 
                                lat=lat_col, 
                                lon=lon_col, 
                                hover_name="title" if "title" in df.columns else None, 
                                color="price_type" if "price_type" in df.columns else None,
                                zoom=11, 
                                height=600)
        
        fig.update_layout(mapbox_style="open-street-map")
        fig.update_layout(title="Carte des Événements à Paris")
        fig.show()
    else:
        print("Coordonnées géographiques non trouvées (lat/lon).")

## 5. Analyse des Prix et de l'Accessibilité

Comparons les événements gratuits et payants. Nous cherchons des colonnes comme `price_type`, `price_detail` ou `access_type`.

In [9]:
if not df.empty:
    price_col_candidates = ['price_type', 'access_type', 'price_detail']
    price_col = next((c for c in price_col_candidates if c in df.columns), None)

    if price_col:
        plt.figure(figsize=(10, 6))
        # Get top 5 price types to avoid clutter
        top_prices = df[price_col].value_counts().head(5).index
        sns.countplot(data=df[df[price_col].isin(top_prices)], y=price_col, order=top_prices, palette='viridis')
        plt.title(f"Distribution des Types de Prix ({price_col})")
        plt.show()
    else:
        print("Aucune colonne explicite sur le type de prix trouvée.")

## 6. Analyse Textuelle et Mots-Clés

Extraction des mots les plus fréquents dans les titres et les descriptions pour comprendre les thématiques populaires.

In [10]:
from collections import Counter
import re

def clean_text(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'[^a-zA-Zà-ü\s]', '', text) # Keep french accents
    return text

if not df.empty:
    text_col = 'title' if 'title' in df.columns else ('title_fr' if 'title_fr' in df.columns else None)

    if text_col:
        all_text = " ".join(df[text_col].dropna().apply(clean_text))
        words = all_text.split()
        
        # Remove simple stop words
        stop_words = set(['de', 'la', 'le', 'et', 'les', 'des', 'en', 'un', 'une', 'du', 'au', 'pour', 'sur', 'a', 'à', 'par', 'dans', 'ce', 'qui'])
        filtered_words = [w for w in words if w not in stop_words and len(w) > 2]
        
        word_counts = Counter(filtered_words).most_common(20)
        
        # Plot
        words_df = pd.DataFrame(word_counts, columns=['Word', 'Count'])
        
        plt.figure(figsize=(12, 6))
        sns.barplot(data=words_df, x='Count', y='Word', palette='magma')
        plt.title("Top 20 des Mots dans les Titres")
        plt.show()
    else:
        print("Colonne de texte non trouvée.")

## Conclusion

Ce notebook a permis de mettre en évidence :
1. **La qualité des données** (colonnes manquantes, format des dates).
2. **La dynamique temporelle** des événements.
3. **La répartition géographique**.
4. **Les thématiques principales** via l'analyse de texte.

Ces insights sont essentiels pour orienter les futures étapes de modélisation ou de création de tableau de bord.

# Analyse Exploratoire des Données (EDA) - Événements à Paris

Ce notebook présente une analyse exploratoire des données récupérées depuis l'API OpenData de Paris concernant les événements "Que faire à Paris ?".

L'objectif est de comprendre la structure des données, de nettoyer les informations et de visualiser quelques tendances clés.

In [11]:
# Install necessary libraries if not already installed
%pip install pandas matplotlib seaborn plotly requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configuration
sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)

## 1. Récupération des Données
Nous allons récupérer un échantillon de données directement depuis l'API pour l'analyse locale.

In [13]:
def fetch_data(limit=1000):
    """
    Fetches data from the OpenData Paris API.
    """
    base_url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/que-faire-a-paris-/records"
    params = {
        'limit': limit,
        'offset': 0
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        return data['results']
    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

# Fetch data sample
print("Fetching data from API...")
events_data = fetch_data(limit=1000)
df = pd.DataFrame(events_data)
print(f"Data loaded: {df.shape[0]} rows, {df.shape[1]} columns")

Fetching data from API...
Error fetching data: 400 Client Error: Bad Request for url: https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/que-faire-a-paris-/records?limit=1000&offset=0
Data loaded: 0 rows, 0 columns


## 2. Aperçu des Données
Inspectons les premières lignes et la structure du dataset pour comprendre les colonnes disponibles.

In [14]:
# Display first rows
df.head()

""


In [15]:
# Display dataset information (types, missing values)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


## 3. Nettoyage des Données
Nous allons convertir les colonnes de dates au bon format et vérifier les doublons.

In [16]:
# Convert date columns to datetime objects
date_cols = ['date_start', 'date_end']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Remove duplicates if any
if duplicates > 0:
    df.drop_duplicates(inplace=True)

Number of duplicate rows: 0


## 4. Analyse des Prix
Regardons la répartition des types de prix (gratuit, payant, etc.).

In [17]:
if 'price_type' in df.columns:
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x='price_type', order=df['price_type'].value_counts().index)
    plt.title("Distribution des Types de Prix")
    plt.xlabel("Type de Prix")
    plt.ylabel("Nombre d'événements")
    plt.show()
else:
    print("Column 'price_type' not found.")

Column 'price_type' not found.


## 5. Analyse Temporelle
Observons la répartition des événements dans le temps (par mois/année).

In [18]:
if 'date_start' in df.columns:
    # Extract month and year
    df['period'] = df['date_start'].dt.to_period('M')
    
    # Count events per period
    period_counts = df['period'].value_counts().sort_index()
    
    plt.figure(figsize=(14, 6))
    period_counts.plot(kind='bar', color='skyblue')
    plt.title("Nombre d'événements par Mois")
    plt.xlabel("Période")
    plt.ylabel("Nombre d'événements")
    plt.xticks(rotation=45)
    plt.show()

## 6. Analyse Géographique
Si les coordonnées sont disponibles, nous pouvons voir où se situent les événements (top lieux).

In [19]:
if 'address_city' in df.columns:
    top_cities = df['address_city'].value_counts().head(10)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x=top_cities.values, y=top_cities.index, palette="viridis")
    plt.title("Top 10 des Villes (ou codes postaux) des Événements")
    plt.xlabel("Nombre d'événements")
    plt.ylabel("Ville")
    plt.show()